In [22]:
import numpy as np #import numpy
import pandas as pd #import pandas
import seaborn as sns # import seaborn
import matplotlib.pyplot as plt #import pyplot
from scipy.stats import pearsonr #for pearson's correlation

from sklearn.model_selection import train_test_split #for splitting the data in train and test
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler #for various scaling methods
from sklearn.linear_model import LogisticRegression #for LogisticRegression
from sklearn.naive_bayes import GaussianNB #for NaiveBayes
from sklearn.neighbors import KNeighborsClassifier #for KNN
from sklearn.svm import SVC #for Support vector classifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score,confusion_matrix,recall_score #for accuracy matrices
from sklearn.metrics import precision_score,classification_report,roc_auc_score,precision_score #for accuracy matrices


In [3]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import matplotlib.style as style
%matplotlib inline
style.use('ggplot')

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter("ignore")

pd.pandas.set_option('display.max_columns',None)

In [4]:
df = pd.read_csv('classification_dataset.csv')
df.head()

,density_per_km,latitude,longitude,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,year,month,Target
0,12625.800781,-34.603684,-58.381559,24,7,0.0,11.6,5,87,6,14,15,14,16,0,59,0.0,1018,24,10,189,10,2016,3,1
1,12625.800781,-34.603684,-58.381559,24,7,0.0,11.6,5,87,6,14,15,14,16,0,59,0.0,1018,24,10,189,10,2016,3,0
2,12625.800781,-34.603684,-58.381559,24,7,0.0,11.6,5,87,6,14,15,14,16,0,59,0.0,1018,24,10,189,10,2016,3,1
3,12625.800781,-34.603684,-58.381559,24,7,0.0,11.6,5,87,6,14,15,14,16,0,59,0.0,1018,24,10,189,10,2016,3,0
4,12625.800781,-34.603684,-58.381559,22,10,0.0,11.5,5,76,10,14,16,14,27,34,72,0.0,1025,22,10,99,18,2016,3,0


In [5]:
df.columns

Index(['density_per_km', 'latitude', 'longitude', 'maxtempC', 'mintempC',
       'totalSnow_cm', 'sunHour', 'uvIndex', 'moon_illumination', 'DewPointC',
       'FeelsLikeC', 'HeatIndexC', 'WindChillC', 'WindGustKmph', 'cloudcover',
       'humidity', 'precipMM', 'pressure', 'tempC', 'visibility',
       'winddirDegree', 'windspeedKmph', 'year', 'month', 'Target'],
      dtype='object')

In [30]:
df.shape

(155223, 25)

In [29]:
df.Target.value_counts()

0    89843
1    65380
Name: Target, dtype: int64

## Feature Scaling

In [6]:
feature_scale=[feature for feature in df.columns if feature not in ['Target']]

scaler=MinMaxScaler()
scaler.fit(df[feature_scale])

data = pd.concat([df[['Target']].reset_index(drop=True),
                    pd.DataFrame(scaler.transform(df[feature_scale]), columns=feature_scale)],axis=1)

In [7]:
data.head()

,Target,density_per_km,latitude,longitude,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,year,month
0,1,0.505636,0.196981,0.742734,0.641791,0.567164,0.0,0.625806,0.444444,0.87,0.622642,0.619718,0.573770,0.628571,0.164835,0.00,0.559140,0.0,0.602941,0.641791,1.0,0.531429,0.152542,0.333333,0.181818
1,0,0.505636,0.196981,0.742734,0.641791,0.567164,0.0,0.625806,0.444444,0.87,0.622642,0.619718,0.573770,0.628571,0.164835,0.00,0.559140,0.0,0.602941,0.641791,1.0,0.531429,0.152542,0.333333,0.181818
2,1,0.505636,0.196981,0.742734,0.641791,0.567164,0.0,0.625806,0.444444,0.87,0.622642,0.619718,0.573770,0.628571,0.164835,0.00,0.559140,0.0,0.602941,0.641791,1.0,0.531429,0.152542,0.333333,0.181818
3,0,0.505636,0.196981,0.742734,0.641791,0.567164,0.0,0.625806,0.444444,0.87,0.622642,0.619718,0.573770,0.628571,0.164835,0.00,0.559140,0.0,0.602941,0.641791,1.0,0.531429,0.152542,0.333333,0.181818
4,0,0.505636,0.196981,0.742734,0.611940,0.611940,0.0,0.619355,0.444444,0.76,0.698113,0.619718,0.590164,0.628571,0.285714,0.34,0.698925,0.0,0.705882,0.611940,1.0,0.274286,0.288136,0.333333,0.181818


## Feature Selection

In [8]:
X = data[['density_per_km', 'latitude', 'longitude', 'maxtempC', 'mintempC', 'DewPointC','precipMM',
       'FeelsLikeC', 'HeatIndexC', 'WindChillC', 'cloudcover','humidity', 'pressure','winddirDegree', 'month']]

y = data['Target']

## Train Test Split

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.50,random_state=10)

## 1. Logistic Regression

In [10]:
logic_r = LogisticRegression() #Instantiate the LogisticRegression object
logic_r.fit(X_train,y_train) #Call the fit method of logistic regression to train the model or to learn the parameters of model
y_pred = logic_r.predict(X_test) #Predict  

In [11]:
print('\033[1m''Confusion Matrix\n',confusion_matrix(y_test,y_pred)) #for confusion matrix
print('-'*40)
LR_accuracy = accuracy_score(y_test,y_pred)
print('Accuracy of Logistic Regression :{:.2f}'.format(LR_accuracy)) #for accuracy score
print('-'*40)
print('\n Classification Report\n',classification_report(y_test,y_pred)) #for classification report


Confusion Matrix
 [[35362  9651]
 [20936 11663]]
----------------------------------------
Accuracy of Logistic Regression :0.61
----------------------------------------

 Classification Report
               precision    recall  f1-score   support

           0       0.63      0.79      0.70     45013
           1       0.55      0.36      0.43     32599

    accuracy                           0.61     77612
   macro avg       0.59      0.57      0.57     77612
weighted avg       0.59      0.61      0.59     77612



## 2. Naive bayes 

In [42]:
NB = GaussianNB()   #Instantiate the Gaussian Naive bayes 
NB.fit(X_train,y_train) #Call the fit method of NB to train the model or to learn the parameters of model
y_predi = NB.predict(X_test) #Predict 

In [43]:
print('\033[1m''Confusion Matrix\n',confusion_matrix(y_test,y_predi)) #for confusion matrix
print('-'*30)
NB_accuracy = accuracy_score(y_test,y_predi)
print('Accuracy of Naive Bayes :{:.2f}'.format(NB_accuracy)) #for accuracy score
print('-'*30)
print('\n Classification Report\n',classification_report(y_test,y_predi)) #for classification report

Confusion Matrix
 [[26095 18918]
 [13426 19173]]
------------------------------
Accuracy of Naive Bayes :0.58
------------------------------

 Classification Report
               precision    recall  f1-score   support

           0       0.66      0.58      0.62     45013
           1       0.50      0.59      0.54     32599

    accuracy                           0.58     77612
   macro avg       0.58      0.58      0.58     77612
weighted avg       0.59      0.58      0.59     77612



## 3. Support Vector machine

In [17]:
svc = SVC()  #Instantiate SVC
svc.fit(X_train,y_train) #Call the fit method of SVC to train the model or to learn the parameters of model
predicted_svc = svc.predict(X_test) #Predict 

print('\033[1m''Confusion Matrix\n',confusion_matrix(y_test,predicted_svc)) #for confusion matrix
print('-'*30)
SVC_accuracy = accuracy_score(y_test,predicted_svc) #for accuracy score
print('Accuracy of SVC :',SVC_accuracy)
print('-'*30)
print('\n Classification Report\n',classification_report(y_test,predicted_svc)) #for classification report


Confusion Matrix
 [[37362  7651]
 [18979 13620]]
------------------------------
Accuracy of SVC : 0.6568829562438798
------------------------------

 Classification Report
               precision    recall  f1-score   support

           0       0.66      0.83      0.74     45013
           1       0.64      0.42      0.51     32599

    accuracy                           0.66     77612
   macro avg       0.65      0.62      0.62     77612
weighted avg       0.65      0.66      0.64     77612



## 4. Random forest

In [20]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(X_train,y_train)
y_pred=classifier.predict(X_test)

print('\033[1m''Confusion Matrix\n',confusion_matrix(y_test,y_pred))
print('-'*30)
print('Accuracy of SVC :',accuracy_score(y_test,y_pred))
print('-'*30)
print('\n Classification Report\n',classification_report(y_test,y_pred))

Confusion Matrix
 [[31738 13275]
 [14595 18004]]
------------------------------
Accuracy of SVC : 0.6409060454568881
------------------------------

 Classification Report
               precision    recall  f1-score   support

           0       0.68      0.71      0.69     45013
           1       0.58      0.55      0.56     32599

    accuracy                           0.64     77612
   macro avg       0.63      0.63      0.63     77612
weighted avg       0.64      0.64      0.64     77612



## 5. Decision Tree

In [26]:
# Defining the decision tree algorithm
dtree=DecisionTreeClassifier()
dtree.fit(X_train,y_train)
y_pred = dtree.predict(X_test)

print('\033[1m''Confusion Matrix\n',confusion_matrix(y_test,y_pred))
print('-'*30)
print('Accuracy of SVC :',accuracy_score(y_test,y_pred))
print('-'*30)
print('\n Classification Report\n',classification_report(y_test,y_pred))

Confusion Matrix
 [[33191 11822]
 [17708 14891]]
------------------------------
Accuracy of SVC : 0.6195176003710766
------------------------------

 Classification Report
               precision    recall  f1-score   support

           0       0.65      0.74      0.69     45013
           1       0.56      0.46      0.50     32599

    accuracy                           0.62     77612
   macro avg       0.60      0.60      0.60     77612
weighted avg       0.61      0.62      0.61     77612

